*Vlad 3.11.2024*

In [ ]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier, CatBoostClassifier, Pool, cv as catboostCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, cv as lgbmCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from tqdm import tqdm
import torch

import glob
import os

import time

# SETTINGS
pd.set_option("display.max_columns", 600)
test_path = "/kaggle/input/test"

predictions_val = pd.DataFrame()
predictions_test = pd.DataFrame()

## All functions

In [ ]:
# FUNCTIONS
def fast_auc(y_true, y_prob):
    """



    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013



    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += 1 - y_i
        auc += y_i * nfalse
    auc /= nfalse * (n - nfalse)
    return auc


def eval_auc_lgb(y_true, y_pred):
    """

    Fast auc eval function for lgb.

    """

    return "auc", fast_auc(y_true, y_pred), True


class CustomAUC(object):

    @staticmethod
    def get_auc(y_true, y_prob):
        """

        fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013

        """

        y_true = np.asarray(y_true)

        y_true = y_true[np.argsort(y_prob)]

        nfalse = 0

        auc = 0

        n = len(y_true)

        for i in range(n):

            y_i = y_true[i]

            nfalse += 1 - y_i

            auc += y_i * nfalse

        auc /= nfalse * (n - nfalse)

        return auc

    def is_max_optimal(self):

        return True  # greater is better

    def evaluate(self, approxes, target, weight):

        y_true = np.array(target).astype(int)

        approx = approxes[:, 1]

        score = self.get_auc(y_true, approx)

        return score, 1

    def get_final_error(self, error, weight):

        return error


def get_time():
    return time.ctime().replace(" ", "_").replace("__", "_").replace(":", "_")


def preprocess(data, is_test=False):

    cors_high = [
        "feature_424",
        "feature_285",
        "feature_141",
        "feature_285",
        "feature_48",
        "feature_178",
        "feature_285",
        "feature_381",
        "feature_239",
        "feature_158",
        "feature_103",
        "feature_208",
        "feature_9",
        "feature_428",
        "feature_323",
        "feature_410",
        "feature_98",
        "feature_285",
        "feature_9",
        "feature_379",
        "feature_495",
        "feature_323",
        "feature_98",
    ]

    data = data.drop(columns=cors_high)

    data["feature_47_div_feature_133"] = data.feature_47 / (data.feature_133 + 1e-8)

    data["feature_47_div_feature_253"] = data["feature_47"] / (data["feature_253"] + 1e-8)

    data["feature_459_div_feature_166"] = data["feature_459"] / (data["feature_166"] + 1e-8)

    data["feature_173_div_feature_467"] = data["feature_173"] / (data["feature_467"] + 1e-8)

    data["feature_467_div_feature_343"] = data["feature_467"] / (data["feature_343"] + 1e-8)

    data["feature_201_div_feature_47"] = data["feature_201"] / (data["feature_47"] + 1e-8)

    data["feature_87_div_feature_492"] = data["feature_87"] / (data["feature_492"] + 1e-8)

    data["feature_201_div_feature_467"] = data["feature_201"] / (data["feature_467"] + 1e-8)

    data["feature_83_div_feature_47"] = data["feature_83"] / (data["feature_47"] + 1e-8)

    data["feature_423_div_feature_87"] = data["feature_423"] / (data["feature_87"] + 1e-8)

    data["id_scaled"] /= data["id"].max()
    
    need_take = [
        "id_scaled",
        "feature_5",
        "feature_11",
        "feature_12",
        "feature_14",
        "feature_18",
        "feature_29",
        "feature_31",
        "feature_35",
        "feature_37",
        "feature_42",
        "feature_45",
        "feature_47",
        "feature_50",
        "feature_53",
        "feature_66",
        "feature_68",
        "feature_80",
        "feature_81",
        "feature_86",
        "feature_87",
        "feature_97",
        "feature_100",
        "feature_106",
        "feature_114",
        "feature_119",
        "feature_131",
        "feature_133",
        "feature_138",
        "feature_139",
        "feature_147",
        "feature_151",
        "feature_153",
        "feature_154",
        "feature_157",
        "feature_161",
        "feature_164",
        "feature_167",
        "feature_168",
        "feature_172",
        "feature_174",
        "feature_186",
        "feature_194",
        "feature_195",
        "feature_197",
        "feature_205",
        "feature_230",
        "feature_234",
        "feature_244",
        "feature_246",
        "feature_247",
        "feature_251",
        "feature_253",
        "feature_259",
        "feature_262",
        "feature_264",
        "feature_265",
        "feature_270",
        "feature_272",
        "feature_318",
        "feature_325",
        "feature_331",
        "feature_332",
        "feature_336",
        "feature_353",
        "feature_364",
        "feature_365",
        "feature_371",
        "feature_383",
        "feature_386",
        "feature_388",
        "feature_390",
        "feature_412",
        "feature_421",
        "feature_439",
        "feature_449",
        "feature_451",
        "feature_452",
        "feature_454",
        "feature_462",
        "feature_467",
        "feature_470",
        "feature_490",
        "feature_498",
    ]

    if "target" in data.columns:
        need_take.append("target")
    if is_test:
        need_take.append("id")
    
    data = data[need_take]
    return data


def train_model(X_train, X_val, X_test, y_train, y_val, model_type, model_params):

    model = None

    if model_type == "cat":
        X_train, X_local_val, y_train, y_local_val = train_test_split(
            X_train, y_train, stratify=y_train, train_size=0.85, shuffle=False
        )

        train_set = Pool(data=X_train, label=y_train)
        eval_set = Pool(data=X_local_val, label=y_local_val)

        model = CatBoostClassifier(**model_params)
        model.fit(train_set, eval_set=eval_set, use_best_model=True, verbose=200)

        oof_prediction = model.predict_proba(X_val)[:, 1]
        test_prediction = model.predict_proba(X_test)[:, 1]

    # elif model_type == "lgbm":
    #     model = LGBMClassifier(device_type="gpu", n_jobs=-1)
    #     model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])

    #     oof_prediction = model.predict_proba(X_val, n_estimatorsation=model.best_iteration_)[:, 1]
    #     test_prediction = model.predict_proba(X_test, n_estimatorsation=model.best_iteration_)[:, 1]

    #     print(model.evals_result_)

    elif model_type == "xgb":
        model = XGBClassifier(**model_params)
        X_train, X_local_val, y_train, y_local_val = train_test_split(
            X_train, y_train, stratify=y_train, train_size=0.85, shuffle=False
        )

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_local_val, y_local_val)],
        )
        
        oof_prediction = model.predict_proba(X_val)[:, 1]
        test_prediction = model.predict_proba(X_test)[:, 1]

    else:
        raise RuntimeError("Wrong model_type is provided.\n")

    return oof_prediction, test_prediction, model


def save_model(model, model_type, file_idx, model_idx):

    try:
        if model_type == "cat":
            save_path = f"CATBOOST_{file_idx}_{model_idx}_{get_time()}"
            model.save_model(save_path, format="cbm")
        elif model_type == "lgbm":
            save_path = f"LGMB_{file_idx}_{model_idx}_{get_time()}"
            model.save_model(save_path, n_estimatorsation=model.best_iteration)
        elif model_type == "xgb":
            save_path = f"XGBOOST_{file_idx}_{model_idx}_{get_time()}"
            model.save_model(save_path)

    except:

        print(f"No save completed for model {model_type}_{model_idx}\tfile{file_idx}")
    finally:
        print(f"{get_time()} SAVE_MODEL COMPLETED")


def get_metadata_from_files(X_val, X_test, y_val, models_list, types_of_models=2, train_files=None):

    if any(not os.path.isfile(train_file) for train_file in train_files):

        raise RuntimeError("Provide correct train_files. " + "Current is {}".format(train_files))

    train_files = sorted(train_files)

    for file_idx, filename in enumerate(train_files):

        print("File: {}".format(filename))
        print("Started at: {}".format(time.ctime()))

        s = time.time()
        X_train = pd.read_csv(filename)
        e = time.time()
        print("{} seconds to read a file".format(e - s))

        X_train = preprocess(X_train)
        y_train = X_train.pop("target")

        for model_idx, (model_type, model_params) in enumerate(models_list):
            model = None
            print(model_type, "-", model_idx)

            # IT MUST BE PROBAS
            train_prediction, test_prediction, model = train_model(
                X_train, X_val, X_test, y_train, y_val, model_type, model_params
            )

            predictions_val = pd.concat((predictions_val, train_prediction), axis=1)
            predictions_test = pd.concat((predictions_test, test_prediction), axis=1)

            try:
                print("no save.")
                # save_model(model, model_type, file_idx, model_idx)
            except Exception as e:
                print("Something went wrong with saving")
                print(e.what())
            finally:
                print("\tModel: {}_{}".format(models_list[model_idx][0], model_idx))

        del X_train

    train_metadata /= len(models_list)
    test_metadata /= len(models_list)

    return train_metadata, test_metadata


def predict_test(X_test, ids, model, test_path=test_path, name=None):
    if name is None:
        name = f"UNTITLED"

    dt = pd.DataFrame({"id": ids, "target": model.predict_proba(X_test)[:, 1]})
    dt.to_csv(f"submission_{name}_{get_time()}.csv", index=False)

## Params

In [ ]:
SEED = 42
SEEDS = [42, 1337, 777, 666, 999]

catboost_params_0 = {
    "verbose": False,
    "max_depth": 5,
    "l2_leaf_reg": 10,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "early_stopping_rounds": 50,
    "task_type": "GPU" if torch.cuda.is_available() else "CPU",
    "random_state": SEEDS[0]

}

catboost_params_1 = {
    "verbose": False,
    "max_depth": 4,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "early_stopping_rounds": 50,
    "task_type": "GPU" if torch.cuda.is_available() else "CPU",
    "random_state": SEEDS[1],
}

catboost_params_2 = {
    "verbose": False,
    "max_depth": 3,
    "l2_leaf_reg": 2,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "early_stopping_rounds": 50,
    "task_type": "GPU" if torch.cuda.is_available() else "CPU",
    "random_state": SEEDS[2],
}

catboost_params_3 = {
    "verbose": False,
    "max_depth": 6,
    "l2_leaf_reg": 5,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "early_stopping_rounds": 50,
    "task_type": "GPU" if torch.cuda.is_available() else "CPU",
    "random_state": SEEDS[3],
}

catboost_params_4 = {
    "verbose": False,
    "max_depth": 8,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "early_stopping_rounds": 50,
    "task_type": "GPU" if torch.cuda.is_available() else "CPU",
    "random_state": SEEDS[4],
    "l2_leaf_reg": 6,
}

catboost_params_5 = {
    "verbose": False,
    "max_depth": 6,
    "loss_function": "Logloss",
    "l2_leaf_reg": 2,
    "eval_metric": "AUC",
    "early_stopping_rounds": 50,
    "task_type": "GPU" if torch.cuda.is_available() else "CPU",
}

xgb_params_0 = {
    "num_iter": 1000,
    "early_stopping_rounds":25,
    "device": "gpu" if torch.cuda.is_available() else "cpu", 
    "max_depth": 7,
    "verbose": 100,
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "seed": SEEDS[0],
    "verbose": 100, 
    "gamma": 0.15,
    "scale_positive_rate": 1,
    "verbosity": 1
}

xgb_params_1 = {
    "num_iter": 1000,
    "learning_rate": 0.05,
    "early_stopping_rounds":25,
    "device": "gpu" if torch.cuda.is_available() else "cpu", 
    "max_depth": 6,
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "seed": SEEDS[1],
    "verbosity": 1,
    "scale_positive_rate": 1,
}

xgb_params_2 = {
    "num_iter": 1000,
    "learning_rate": 0.05,
    "early_stopping_rounds":25,
    "device": "gpu" if torch.cuda.is_available() else "cpu", 
    "max_depth": 3,
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "seed": SEEDS[2], 
    "verbosity": 1,
    "scale_positive_rate": 1,
}

xgb_params_3 = {
    "num_iter": 1000,
    "early_stopping_rounds":25,
    "device": "gpu" if torch.cuda.is_available() else "cpu", 
    "max_depth": 5,
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "seed": SEEDS[3],
    "verbosity": 1,
    "min_child_weight":1,
    "learning_rate": 0.05
}

xgb_params_4 = {
    "num_iter": 1000, 
    "early_stopping_rounds":25, 
    "device": "gpu" if torch.cuda.is_available() else "cpu", 
    "max_depth": 3,
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "seed": SEEDS[4],
    "verbosity": 1
}

## Models to train: CatBoost, LGBM, XGB

In [ ]:
models_list = [
    ["xgb", xgb_params_0],
    ["xgb", xgb_params_1],
    ["xgb", xgb_params_2],
    ["xgb", xgb_params_3],
    ["xgb", xgb_params_4],

    ["cat", catboost_params_0],
    ["cat", catboost_params_1],
    ["cat", catboost_params_2],
    ["cat", catboost_params_3],
    ["cat", catboost_params_4]
]

## Validation:: train_10



## Test: concatenated test files

In [ ]:
val_file = "/kaggle/input/trainset/train_10.csv"

s = time.time()
X_val = pd.read_csv(val_file)
y_val = X_val.pop('target')
X_val = preprocess(X_val)
e = time.time()

print("{} seconds to preprocess a file {}".format(e - s, val_file))

def read_test(test_path=test_path):
    X_test = None
    for test_file in tqdm(os.listdir(test_path)):
        print(test_file)
        if X_test is None:
            X_test = pd.read_csv(test_path + '/' + test_file)
        else:
            X_test = pd.concat((X_test, pd.read_csv(test_path + '/' + test_file)))
    return preprocess(X_test, is_test=True)

s = time.time()
X_test = read_test(test_path)
test_id = X_test.pop('id')
e = time.time()

print("{} seconds to preprocess test files".format(e - s))

65.71126103401184 seconds to preprocess a file /kaggle/input/trainset/train_10.csv


  0%|          | 0/10 [00:00<?, ?it/s]

test_4.csv


 10%|█         | 1/10 [00:07<01:03,  7.09s/it]

test_1.csv


...

test_2.csv


 90%|█████████ | 9/10 [01:07<00:07,  7.66s/it]

test_6.csv


100%|██████████| 10/10 [01:14<00:00,  7.50s/it]


75.66444516181946 seconds to preprocess test files


## Are params correct?

In [ ]:
train_files = [
    "/kaggle/input/train_1.csv",
    "/kaggle/input/trainset/train_2.csv",
    "/kaggle/input/trainset/train_3.csv",
    "/kaggle/input/trainset/train_4.csv",
    "/kaggle/input/trainset/train_5.csv",
    "/kaggle/input/trainset/train_6.csv", 
    "/kaggle/input/trainset/train_7.csv", 
    "/kaggle/input/trainset/train_8.csv", 
    "/kaggle/input/trainset/train_9.csv"
]

train_metadata, test_metadata = get_metadata_from_files(
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    models_list=models_list,
    train_files=train_files,
    types_of_models=2
)

File: /kaggle/input/train_1.csv
Started at: Thu Nov  7 06:48:39 2024
63.559215784072876 seconds to read a file
lgbm - 0
[LightGBM] [Info] Number of positive: 5187, number of negative: 443859
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 19526
[LightGBM] [Info] Number of data points in the train set: 449046, number of used features: 83
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 82 dense feature groups (35.97 MB) transferred to GPU in 0.028937 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011551 -> initscore=-4.449351
[LightGBM] [Info] Start training from score -4.449351
{'training': OrderedDict([('binary_logloss', [0.06007244570021198, 0.05856468855016451, 0.05749559939232709, 0.056671264273876684, 0.05586176381327535, 0.055209227322484275, 0.054670222455626555, 0.05420325113126847, 0.05379228248078476, 0.05339103799815248, 0.05303091346268185, 0.05268280133203866, 0.05238866853411001, 0.0521025051105913, 0.051839537806421865, 0.05159218273292958, 0.05133829295134498, 0.0510974782980123, 0.05087122399546809, 0.05064362373659835, 0.050422260506018425, 0.05018466462185024, 0.04996962658414863, 0.04974853404816797, 0.049507107494229055, 0.04932361713167617, 0.049158713778863605, 0.04897206282825913, 0.048785885273532

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:50:42] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_iter", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[3]	validation_0-auc:0.82378	validation_1-auc:0.72374
[4]	validation_0-auc:0.83071	validation_1-auc:0.72469
[5]	validation_0-auc:0.83631	validation_1-auc:0.72747
[6]	validation_0-auc:0.84581	validation_1-auc:0.72808
[7]	validation_0-auc:0.85260	validation_1-auc:0.72905
[8]	validation_0-auc:0.85875	validation_1-auc:0.72746
[9]	validation_0-auc:0.86589	validation_1-auc:0.72763
[10]	validation_0-auc:0.87329	validation_1-auc:0.72818
[11]	validation_0-auc:0.88145	validation_1-auc:0.72715
[12]	validation_0-auc:0.88809	validation_1-auc:0.72696
[13]	validation_0-auc:0.89416	validation_1-auc:0.72783
[14]	validation_0-auc:0.89857	validation_1-auc:0.73259
[15]	validation_0-auc:0.90350	validation_1-auc:0.73194
[16]	validation_0-auc:0.90843	validation_1-auc:0.73303
[17]	validation_0-auc:0.91420	validation_1-auc:0.73283
[18]	validation_0-auc:0.91756	validation_1-auc:0.73617
[19]	validation_0-auc:0.92176	validation_1-auc:0.73871
[20]	validation_0-auc:0.92605	validation_1-auc:0.73691
[21]	validation_0

...

200:	test: 0.8463527	best: 0.8463527 (200)	total: 1.84s	remaining: 7.33s
400:	test: 0.8503869	best: 0.8503869 (400)	total: 3.7s	remaining: 5.53s
600:	test: 0.8519023	best: 0.8519023 (600)	total: 5.53s	remaining: 3.67s
800:	test: 0.8527481	best: 0.8527481 (800)	total: 7.39s	remaining: 1.83s
999:	test: 0.8533550	best: 0.8533550 (999)	total: 9.2s	remaining: 0us
bestTest = 0.8533549607
bestIteration = 999
no save.
	Model: cat_17
cat - 18
Learning rate set to 0.042372
0:	test: 0.7644397	best: 0.7644397 (0)	total: 11.9ms	remaining: 11.9s


Default metric period is 5 because AUC is/are not implemented for GPU


200:	test: 0.8480157	best: 0.8480157 (200)	total: 2.22s	remaining: 8.82s
400:	test: 0.8512222	best: 0.8512222 (400)	total: 4.36s	remaining: 6.51s
600:	test: 0.8525860	best: 0.8525918 (599)	total: 6.47s	remaining: 4.3s
800:	test: 0.8534087	best: 0.8534087 (800)	total: 8.57s	remaining: 2.13s
999:	test: 0.8536993	best: 0.8537075 (984)	total: 10.7s	remaining: 0us
bestTest = 0.8537074625
bestIteration = 984
Shrink model to first 985 iterations.
no save.
	Model: cat_18
cat - 19
Learning rate set to 0.042372
0:	test: 0.7153858	best: 0.7153858 (0)	total: 9.03ms	remaining: 9.02s


Default metric period is 5 because AUC is/are not implemented for GPU


200:	test: 0.8424882	best: 0.8424882 (200)	total: 1.62s	remaining: 6.46s
400:	test: 0.8476569	best: 0.8476569 (400)	total: 3.22s	remaining: 4.81s
600:	test: 0.8499862	best: 0.8499862 (600)	total: 4.82s	remaining: 3.2s
800:	test: 0.8510059	best: 0.8510118 (797)	total: 6.37s	remaining: 1.58s
999:	test: 0.8516417	best: 0.8516417 (999)	total: 7.99s	remaining: 0us
bestTest = 0.8516416848
bestIteration = 999
no save.
	Model: cat_19


In [7]:
train_metadata.shape, test_metadata.shape

((449054, 9), (509532, 9))

In [8]:
print(X_val.shape, X_test.shape)

print(train_metadata.shape, test_metadata.shape)

(449054, 83) (509532, 83)
(449054, 9) (509532, 9)


In [9]:
print(X_val.shape, X_test.shape)
X_OOF_train = np.hstack((X_val, train_metadata))
X_OOF_test = np.hstack((X_test, test_metadata))
print(X_OOF_train.shape, X_OOF_test.shape, X_OOF_test.shape)

(449054, 83) (509532, 83)
(449054, 92) (509532, 92) (509532, 92)


In [10]:
pd.DataFrame(X_OOF_train).to_csv("X_OOF_TRAIN.csv")
pd.DataFrame(X_OOF_test).to_csv("X_OOF_TEST.csv")

In [11]:
pd.DataFrame(train_metadata).to_csv("TRAIN_META.csv")
pd.DataFrame(test_metadata).to_csv("TEST_META.csv")

# Predictions on metadata



## Catboost: no cv

In [12]:
try:
    catboost_meta = CatBoostClassifier(
        iterations=200,
        eval_metric="AUC",
        task_type="GPU" if torch.cuda.is_available() else "CPU",
        verbose=10
    )
    
    catboost_meta.fit(train_metadata, y_val)
    predict_test(test_metadata, test_id, catboost_meta, name="catboost_meta_raw")

    print(catboost_meta.feature_importances_)
except Exception as e:
    print("oops")

Learning rate set to 0.1019
0:	total: 8.68ms	remaining: 1.73s
10:	total: 80.9ms	remaining: 1.39s
20:	total: 151ms	remaining: 1.28s


Default metric period is 5 because AUC is/are not implemented for GPU


30:	total: 221ms	remaining: 1.21s
40:	total: 285ms	remaining: 1.1s
50:	total: 348ms	remaining: 1.02s
60:	total: 411ms	remaining: 937ms
70:	total: 474ms	remaining: 862ms
80:	total: 537ms	remaining: 789ms
90:	total: 600ms	remaining: 718ms
100:	total: 667ms	remaining: 653ms
110:	total: 737ms	remaining: 591ms
120:	total: 806ms	remaining: 526ms
130:	total: 876ms	remaining: 461ms
140:	total: 946ms	remaining: 396ms
150:	total: 1.01s	remaining: 329ms
160:	total: 1.08s	remaining: 261ms
170:	total: 1.14s	remaining: 194ms
180:	total: 1.21s	remaining: 127ms
190:	total: 1.28s	remaining: 60.2ms
199:	total: 1.34s	remaining: 0us
[11.23751716 28.4070032   4.5984258   6.12504714  7.39790304 12.9354651
 10.42783655  4.91734657 13.95345544]


## Catboost: meta + small cv

In [13]:
from sklearn.metrics import roc_auc_score
try:
    catboost_cv = {
        "max_depth": [3, 5, 8],
        'l2_leaf_reg': [1, 16, 32],
    }
    
    catboost_meta = CatBoostClassifier(
        iterations=500,
        eval_metric="AUC",
        task_type="GPU" if torch.cuda.is_available() else "CPU",
        verbose=100
    )
    catboost_meta_params, cv_results = catboost_meta.randomized_search(catboost_cv, X=train_metadata, y=y_val, cv=3, n_iter=6, plot=True)
    
    print(cv_results)
    print("Best params for catboost metamodel\n")
    print(catboost_meta_params)
    predict_test(test_metadata, test_id, catboost_meta, name="catboost_metamodel_cv")
except Exception as e:
    print("oops... GB...")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7506196	best: 0.7506196 (0)	total: 29.7ms	remaining: 14.8s
100:	test: 0.8495238	best: 0.8495238 (100)	total: 2.45s	remaining: 9.68s
200:	test: 0.8510597	best: 0.8510597 (200)	total: 4.9s	remaining: 7.28s
300:	test: 0.8515124	best: 0.8515124 (300)	total: 7.33s	remaining: 4.85s
400:	test: 0.8518216	best: 0.8518216 (400)	total: 9.78s	remaining: 2.41s
499:	test: 0.8521015	best: 0.8521015 (495)	total: 12.2s	remaining: 0us
bestTest = 0.8521015346
bestIteration = 495
Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	loss: 0.8521015	best: 0.8521015 (0)	total: 12.5s	remaining: 1m 2s
0:	test: 0.7506196	best: 0.7506196 (0)	total: 25.4ms	remaining: 12.7s


Default metric period is 5 because AUC is/are not implemented for GPU


100:	test: 0.8490439	best: 0.8490439 (100)	total: 2.46s	remaining: 9.71s
200:	test: 0.8507662	best: 0.8507662 (200)	total: 4.88s	remaining: 7.26s
300:	test: 0.8514408	best: 0.8514408 (300)	total: 7.31s	remaining: 4.83s
400:	test: 0.8518414	best: 0.8518414 (400)	total: 9.78s	remaining: 2.41s
499:	test: 0.8521311	best: 0.8521311 (495)	total: 12.1s	remaining: 0us
bestTest = 0.8521311283
bestIteration = 495
Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
1:	loss: 0.8521311	best: 0.8521311 (1)	total: 24.7s	remaining: 49.5s
0:	test: 0.7506196	best: 0.7506196 (0)	total: 25.7ms	remaining: 12.8s


...

100:	test: 0.8496810	best: 0.8496810 (100)	total: 3.84s	remaining: 15.2s
200:	test: 0.8514522	best: 0.8514522 (200)	total: 7.65s	remaining: 11.4s
300:	test: 0.8520523	best: 0.8520523 (300)	total: 11.4s	remaining: 7.54s
400:	test: 0.8523779	best: 0.8523779 (400)	total: 15.1s	remaining: 3.72s
499:	test: 0.8525471	best: 0.8525471 (499)	total: 18.6s	remaining: 0us
bestTest = 0.8525471389
bestIteration = 499
Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
5:	loss: 0.8525471	best: 0.8525471 (5)	total: 1m 32s	remaining: 0us
Estimating final quality...


Default metric period is 5 because AUC is/are not implemented for GPU


Training on fold [0/3]
0:	test: 0.8297152	best: 0.8297152 (0)	total: 40.6ms	remaining: 20.3s
100:	test: 0.8567487	best: 0.8567487 (100)	total: 3.85s	remaining: 15.2s
200:	test: 0.8574694	best: 0.8574694 (200)	total: 7.64s	remaining: 11.4s
300:	test: 0.8577845	best: 0.8577845 (300)	total: 11.3s	remaining: 7.44s
400:	test: 0.8579164	best: 0.8579164 (400)	total: 14.7s	remaining: 3.62s
499:	test: 0.8579804	best: 0.8579804 (490)	total: 17.9s	remaining: 0us
bestTest = 0.8579804301
bestIteration = 490
Training on fold [1/3]
0:	test: 0.8157281	best: 0.8157281 (0)	total: 39.8ms	remaining: 19.9s
100:	test: 0.8560943	best: 0.8560943 (100)	total: 3.86s	remaining: 15.2s
200:	test: 0.8567332	best: 0.8567332 (200)	total: 7.7s	remaining: 11.4s
300:	test: 0.8569187	best: 0.8569187 (300)	total: 11.4s	remaining: 7.53s
400:	test: 0.8570693	best: 0.8570693 (400)	total: 15s	remaining: 3.7s
499:	test: 0.8571692	best: 0.8571694 (495)	total: 18.5s	remaining: 0us
bestTest = 0.8571694195
bestIteration = 495
Trai

Default metric period is 5 because AUC is/are not implemented for GPU


cv_results
Best params for catboost metamodel

params


## LogisticRegression: meta

In [14]:
from sklearn.linear_model import LogisticRegression
try:
    logreg = LogisticRegression(max_iter=5000)
    logreg.fit(train_metadata, y_val)
    predict_test(test_metadata, test_id, logreg, name="simple_logreg_metamodel_no_oof")
except Exception as e:
    print("oops.. regression")

# Initial data + model predictions


## Catboost: OOF + CV

In [15]:
from sklearn.metrics import roc_auc_score
try:
    catboost_cv = {
        "max_depth": [3, 5, 10],
        'l2_leaf_reg': [1, 5, 10]
    }
    
    catboost_meta = CatBoostClassifier(
        iterations=1000,
        eval_metric="AUC",
        task_type="GPU" if torch.cuda.is_available() else "CPU",
        verbose=50
    )
    
    catboost_meta_params, cv_results = catboost_meta.randomized_search(catboost_cv, X=X_OOF_train, y=y_val, cv=3, n_iter=8, plot=True)
    
    print(cv_results)
    print("Best params for catboost OOF metamodel\n")
    print(catboost_meta_params)
    predict_test(X_OOF_test, test_id, catboost_meta, name="catboost_OOF_cv")
except Exception as e:
    print("no catboost oof cv submission...")

Default metric period is 5 because AUC is/are not implemented for GPU


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7338865	best: 0.7338865 (0)	total: 34.7ms	remaining: 34.7s
50:	test: 0.8494048	best: 0.8494048 (50)	total: 1.47s	remaining: 27.3s
100:	test: 0.8520179	best: 0.8520179 (100)	total: 2.86s	remaining: 25.5s
150:	test: 0.8526740	best: 0.8526740 (150)	total: 4.3s	remaining: 24.2s
200:	test: 0.8531814	best: 0.8531814 (200)	total: 5.69s	remaining: 22.6s
250:	test: 0.8534147	best: 0.8534155 (245)	total: 7.09s	remaining: 21.2s
300:	test: 0.8535358	best: 0.8535445 (285)	total: 8.51s	remaining: 19.8s
350:	test: 0.8536720	best: 0.8536720 (350)	total: 9.89s	remaining: 18.3s
400:	test: 0.8538398	best: 0.8538398 (400)	total: 11.2s	remaining: 16.8s
450:	test: 0.8539409	best: 0.8539409 (450)	total: 12.6s	remaining: 15.3s
500:	test: 0.8540027	best: 0.8540115 (490)	total: 14s	remaining: 13.9s
550:	test: 0.8540917	best: 0.8540917 (550)	total: 15.4s	remaining: 12.5s
600:	test: 0.8541397	best: 0.8541430 (595)	total: 16.7s	remaining: 11.1s
650:	test: 0.8542037	best: 0.8542037 (650)	total: 18.1s	rem

Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.8195504	best: 0.8195504 (0)	total: 691ms	remaining: 11m 30s
50:	test: 0.8490083	best: 0.8490083 (50)	total: 17.4s	remaining: 5m 24s
100:	test: 0.8527866	best: 0.8527866 (100)	total: 44.2s	remaining: 6m 33s
150:	test: 0.8540104	best: 0.8540104 (150)	total: 1m 13s	remaining: 6m 50s
200:	test: 0.8544533	best: 0.8544533 (200)	total: 1m 46s	remaining: 7m 2s
250:	test: 0.8547922	best: 0.8547922 (250)	total: 2m 16s	remaining: 6m 48s
300:	test: 0.8551722	best: 0.8551722 (300)	total: 2m 49s	remaining: 6m 33s
350:	test: 0.8555119	best: 0.8555119 (350)	total: 3m 21s	remaining: 6m 13s
400:	test: 0.8556864	best: 0.8556914 (395)	total: 3m 53s	remaining: 5m 48s
450:	test: 0.8558429	best: 0.8558429 (450)	total: 4m 26s	remaining: 5m 24s
500:	test: 0.8559144	best: 0.8559144 (500)	total: 5m 1s	remaining: 5m
550:	test: 0.8560900	best: 0.8560900 (550)	total: 5m 35s	remaining: 4m 33s
600:	test: 0.8561717	best: 0.8562115 (595)	total: 6m 10s	remaining: 4m 6s
650:	test: 0.8562546	best: 0.8562584 (64

Default metric period is 5 because AUC is/are not implemented for GPU


Training on fold [0/3]
0:	test: 0.8326741	best: 0.8326741 (0)	total: 698ms	remaining: 11m 37s
50:	test: 0.8548156	best: 0.8548156 (50)	total: 20.9s	remaining: 6m 29s
100:	test: 0.8582785	best: 0.8582785 (100)	total: 49.8s	remaining: 7m 23s
150:	test: 0.8592563	best: 0.8592563 (150)	total: 1m 20s	remaining: 7m 34s
200:	test: 0.8594966	best: 0.8595010 (195)	total: 1m 51s	remaining: 7m 22s
250:	test: 0.8596728	best: 0.8596728 (250)	total: 2m 20s	remaining: 6m 59s
300:	test: 0.8598390	best: 0.8598403 (295)	total: 2m 50s	remaining: 6m 37s
350:	test: 0.8599214	best: 0.8599291 (340)	total: 3m 20s	remaining: 6m 11s
400:	test: 0.8600185	best: 0.8600185 (400)	total: 3m 47s	remaining: 5m 39s
450:	test: 0.8602550	best: 0.8602550 (450)	total: 4m 19s	remaining: 5m 15s
500:	test: 0.8602835	best: 0.8602897 (485)	total: 4m 51s	remaining: 4m 49s
550:	test: 0.8602902	best: 0.8603238 (525)	total: 5m 21s	remaining: 4m 21s
600:	test: 0.8603472	best: 0.8603472 (600)	total: 5m 52s	remaining: 3m 54s
650:	test:

Default metric period is 5 because AUC is/are not implemented for GPU


cv_results
Best params for catboost OOF metamodel

params


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

try:
    logreg_params = {
        "max_iter": [3000, 5000], 
        "penalty": ["l1", "l2", None],
        "tol": [1e-8, 1e-5], 
        "warm_start": [True],
        "solver":["saga", "liblinear"]
    }
    logreg_meta = GridSearchCV(LogisticRegression(), logreg_params, verbose=1, scoring="roc_auc")
    logreg_meta.fit(train_metadata, y_val)
    logreg_metamodel = logreg_meta.best_estimator_
    
    print("Best params for LogReg metamodel\n")
    print(logreg_metamodel.get_params)
    
    print("Logreg metamodel best score: {}".format(logreg_meta.best_score_))
    predict_test(test_metadata, test_id, logreg_metamodel, name="logreg_meta_cv_no_oof")
except Exception as e:
    print(e)
    print("no regression submission...")

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1216, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py", line 1223, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(mult

Best params for LogReg metamodel

<bound method BaseEstimator.get_params of LogisticRegression(max_iter=3000, solver='liblinear', warm_start=True)>
Logreg metamodel best score: 0.8560889523459784
